***PRACTICA 1 B***
Parte B de la práctica 1, para el uso de esta práctica se ha utilizado el command prompt de Anaconda para poder instalar la librería surprise usando el siguiente comando:

conda install -c conda-forge scikit-surprise

***Antes de empezar***: determina un valor para una variable SEED (el que sea). Esta variable se la vamos a pasar a los modelos para que cada vez que se ejecute el código salgan los mismos resultados. De esta forma os aseguráis de que los resultados que os salgan serán los mismos que me salgan a mí al ejecutar el código. Si no hacéis esto, vuestro análisis puede no tener ningún sentido en mi ejecución y os arriesgáis al no apto.

In [ ]:
from surprise import Dataset,SVD
from surprise.model_selection import cross_validate, train_test_split

#Como se nos pide en el enunciado creamos la variable seed que usaremos cuando creemos los modelos.
SEED = 123

1. Carga el dataset de MovieLens de 100K. MovieLens es un dataset muy usado en el desarrollo y evaluación de sistemas de recomendación, y el dataset de 100k contiene 100K interacciones entre usuarios y productos, almacenando qué usuario ha puntuado qué película y con qué rating.

In [ ]:
#Cargamos el dataset de MovieLens-100k desde la funcion de surprise load_builtin().
data = Dataset.load_builtin("ml-100k")
#Comprobamos que el dataset se ha cargado correctamente.
data.raw_ratings

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013'),
 ('62', '257', 2.0, '879372434'),
 ('286', '1014', 5.0, '879781125'),
 ('200', '222', 5.0, '876042340'),
 ('210', '40', 3.0, '891035994'),
 ('224', '29', 3.0, '888104457'),
 ('303', '785', 3.0, '879485318'),
 ('122', '387', 5.0, '879270459'),
 ('194', '274', 2.0, '879539794'),
 ('291', '1042', 4.0, '874834944'),
 ('234', '1184', 2.0, '892079237'),
 ('119', '392', 4.0, '886176814'),
 ('167', '486', 4.0, '892738452'),
 ('299', '144', 4.0, '877881320'),
 ('291', '118', 2.0, '874833878'),
 ('308', '1', 4.0, '887736532'),
 ('95', '546', 2.0, '879196566'),
 ('38', '95', 5.0, '892430094'),
 ('102', '768', 2.0, '883748450'),
 ('63', '277', 4.0, '875747401

2. Divide el dataset en una partición simple, donde el conjunto de entrenamiento sea el 75% de las interacciones y el resto forme parte del conjunto de evaluación. Aquí tenéis que pasarle random_state = SEED a la función.

In [ ]:
'''
Dividimos el dataset entre las particiones de entrenamiento y validacion usando la función surprise.train_test_split().
Como se nos especifica designamos como un 25% del dataset original como el set de testeo.
'''
data_train, data_test = train_test_split(data, test_size=0.25, random_state=SEED)

3. Vamos a evaluar distintos algoritmos de recomendación: 
    a. Filtrado colaborativo basado en vecinos (KNNBasic en la librería: https://surprise.readthedocs.io/en/stable/knn_inspired.html), tanto basado en usuarios como en productos. Utilizaremos la métrica de similitud de Pearson.
    b. Filtrado colaborativo basado en modelos usando factorización de matrices(https://surprise.readthedocs.io/en/stable/matrix_factorization.html), usando los algoritmos de SVD y NMF.
4. Cada uno de estos algoritmos se entrenarán con el conjunto de entrenamiento. Aquí tenéis que pasarle random_state = SEED a cada uno de los modelos.
5. Después se obtendrán las predicciones que todos los algoritmos obtienen para el conjunto de evaluación. Muestra el resultado de 5 predicciones e interpreta los resultados.

***Anotación***: Se ha decidido juntar la respuesta de las tres preguntas para mejor comprensión del código desarrollado

In [ ]:
'''
Para la solución del ejercicio usaremos los modelos KNNBasic, SVD y NMF

KNNBasic es un modelo basado en K-Nearest-Neighbours, un algoritmo de aprendizaje supervisado usado tanto para clasificación como para regresión.

SVD es un modelo basado en la factorización de matrices es una técnica clave en sistemas de recomendación, reducción de dimensionalidad y procesamiento de datos. 
Su objetivo es descomponer una matriz grande y compleja en el producto de matrices más pequeñas para descubrir estructuras latentes en los datos.

NMF funciona de manera similar a SVD con la excepción de que los factores latentes sean no negativos.
'''

from surprise.prediction_algorithms import KNNBasic,SVD,NMF
from surprise import accuracy
from sklearn.metrics import precision_score, recall_score, ndcg_score

'''En esta estructura almacenamos los diferentes modelos que vamos a utilizar'''
algorithms = {
    "User-based KNN": KNNBasic(sim_options={'name': 'pearson', 'user_based': True}),
    "Item-based KNN": KNNBasic(sim_options={'name': 'pearson', 'user_based': False}),
    "SVD": SVD(random_state=SEED),
    "NMF": NMF(random_state=SEED)
}

# Entrenar y evaluar modelos
results = []
predictions_dict = {}

#En este for vamos a entrenar los diferentes modelos
for name, algo in algorithms.items():
    algo.fit(data_train)
    #Guardamos las predicciones del modelo que estamos evaluando
    predictions = algo.test(data_test)
    predictions_dict[name] = predictions
    # Calcular RMSE de modelo algo
    rmse = accuracy.rmse(predictions, verbose=False)
    
    # Obtener ratings reales y predichos
    y_true = [pred.r_ui for pred in predictions]
    y_pred = [pred.est for pred in predictions]
    
    # Convertir en relevantes (rating >= 4)
    #Se utiliza la condición de >= ya que si utilizasemos > tendríamos pocos datos y los valores de precisión se verían afectados negativamente
    y_true_binary = [1 if r >= 4 else 0 for r in y_true]
    y_pred_binary = [1 if r >= 4 else 0 for r in y_pred]
    
    # Calculamos las métricas
    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    ndcg = ndcg_score([y_true_binary], [y_pred], k=10)
    results.append([name, rmse, precision, recall, ndcg])
    
    


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


6. Crea una tabla con los valores que se obtienen para las métricas de evaluación RMSE, precision, recall, y NDCG (para NDCG, k es el tamaño de la lista de recomendación y será k = 10). Surprise solo implementa RMSE. Las demás las podéis encontrar en sklearn usando precision_score, recall_score, ndcg_score.


In [ ]:
import pandas as pd

# Crear un DataFrame con los resultados que hemos obteido.
results_df = pd.DataFrame(results, columns=["Algoritmo", "RMSE", "Precisión", "Recall", "NDCG"])
print(results_df)

        Algoritmo      RMSE  Precisión    Recall      NDCG
0  User-based KNN  1.011588   0.775630  0.327193  0.615385
1  Item-based KNN  1.036061   0.803561  0.200437  0.904762
2             SVD  0.932241   0.833691  0.367892  0.907407
3             NMF  0.965540   0.820550  0.366873  0.916667


Análisis de los resultados:

Basado en Usuarios
- Es el que peores resultados aporta en cuanto a las metricas de recomendación (precisión, recall y NDCG)

Basado en Productos
- En general, funciona algo mejor que el basado en usuarios aunque tiene un error más grande (RMSE peor).

***Para el desarrollo de sistemas de recomendación el algoritmo más recomendado es el de factorización de matrices, por tanto los mejores candidatos son el SVD y el NMF***

SVD
- Es buena para predecir la calificacion numerica (menor RMSE), pero su ranking (NDCG) no es el mas alto.

NMF
- En general, es el metodo que rinde peor en estas metricas comparado con los otros tres.

Conclusiones:

Si la prioridad es la exactitud numerica de las predicciones (que el sistema acierte con la calificacion lo mas cerca posible a la real), el mejor es SVD porque tiene el RMSE mas bajo.

Si la prioridad es la calidad de la recomendacion en terminos de relevancia (cantidad de items que realmente le gustan al usario), en este caso SVD aporta mejores resultados.

Dado que en la mayoría de escenarios de recomendacion se valoran mas las metricas de relevancia (precision, recall, NDCG) que la exactitud absoluta de, el metodo basado SVD es la mejor eleccion global entre los algoritmos presentados.